<a href="https://colab.research.google.com/github/mmcastillo/al112248/blob/main/3DUnet_patches_aumentados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
drive.mount('/content/drive/');
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras import layers
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.layers import Activation, MaxPool2D, Concatenate
from scipy import ndimage
import random
import natsort
import glob

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [40]:
train_img_dir = '/content/drive/MyDrive/DOCTORADO/python/datos_para_entrenar_modelos/modelo_1.1/train/imgs/';
train_mask_dir = '/content/drive/MyDrive/DOCTORADO/python/datos_para_entrenar_modelos/modelo_1.1/train/masks/';
train_img_list = os.listdir(train_img_dir)
train_img_list = natsort.natsorted(train_img_list)
train_mask_list = os.listdir(train_mask_dir)
train_mask_list = natsort.natsorted(train_mask_list)

val_img_dir = '/content/drive/MyDrive/DOCTORADO/python/datos_para_entrenar_modelos/modelo_1.1/val/imgs/';
val_mask_dir = '/content/drive/MyDrive/DOCTORADO/python/datos_para_entrenar_modelos/modelo_1.1/val/masks/';
val_img_list = os.listdir(val_img_dir)
val_img_list = natsort.natsorted(val_img_list)
val_mask_list = os.listdir(val_mask_dir)
val_mask_list = natsort.natsorted(val_mask_list)

In [41]:
def load_img(img_dir, img_list):
    images=[]
    for i, image_name in enumerate(img_list):    
        if (image_name.split('.')[1] == 'npy'):
            
            image = np.load(img_dir+image_name)
                      
            images.append(image)
    images = np.array(images)
    
    return(images)

In [56]:
#imageloader con batch variable ya que cada paciente cuenta con un número 
#distinto de patches negativos. se extraen 50 negativos y 50 positivos generados
#con data augmentation 

def imageLoader(img_dir, img_list, mask_dir, mask_list):

    L = len(img_list)

    #keras needs the generator infinite, so we will use while true  
    while True:

        temp=[]
        for i in range(len(img_list)):
          if img_list[i].split('_')[0]=='2':
            temp.append(img_list[i])
          batch_size = len(temp)

        batch_start = 0
        batch_end = batch_size
        paciente = 2

        while batch_start < L:
            limit = min(batch_end, L)
            
            #se extrae lista de archivos de cada paciente
            temp=[]
            for i in range(len(img_list)):
              if img_list[i].split('_')[0]==str(paciente):
                temp.append(img_list[i])
                batch_size = len(temp)
            
            list_temp=[]
            for i in range(50):
              inx = random.randint(0,len(temp)-1)
              list_temp.append(temp[inx])

            X = load_img(img_dir, list_temp)
            Y = load_img(mask_dir, list_temp)

            X = X.astype(np.float32)  
            Y = Y.astype(np.float32)

            Z = list_temp

            yield (X,Y,Z) #a tuple with two numpy arrays with batch_size samples
           
            paciente += 1
            batch_start += batch_size   
            batch_end += batch_size

In [ ]:
for paciente in range(2,11):
  temp=[]
  for i in range(len(train_img_list)):
    if train_img_list[i].split('_')[0]==str(paciente):
      temp.append(train_img_list[i])
  #print(len(temp))

  for i in range(5):
    inx = random.randint(0,len(temp))
    print(temp[inx])

In [57]:

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list)

val_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list)


In [86]:
img,mask,Z = train_img_datagen.__next__()
print(img.shape)
print(mask.shape)
print(np.unique(mask))

#Encuentra los patches 3D donde las máscaras contienen valores 1 y 0 (donde hay aneurisams)
if np.unique(mask).shape[0]  == 2:
  for i in range(mask.shape[0]):
    if np.unique(mask[i,:,:,:]).shape[0] == 2: 
      n_patch_aneurisma = i
      print(f'En el patch número {n_patch_aneurisma+1} se encuentra el aneurisma')
      n_imagenes_aneurismas=[]
      for j in range(mask.shape[1]):
        if np.unique(mask[n_patch_aneurisma,j,:,:]).shape[0] == 2:
          print(f'En las imagenes {j+1}')

(50, 64, 64, 64, 1)
(50, 64, 64, 64, 1)
[0.]


In [87]:
print(Z)

['2_A_99.npy', '2_A_76.npy', '2_A_95.npy', '2_A_64.npy', '2_A_114.npy', '2_A_17.npy', '2_A_116.npy', '2_A_47.npy', '2_A_49.npy', '2_A_76.npy', '2_A_109.npy', '2_A_19.npy', '2_A_16.npy', '2_A_50.npy', '2_A_117.npy', '2_A_117.npy', '2_A_128.npy', '2_A_110.npy', '2_A_99.npy', '2_A_114.npy', '2_A_106.npy', '2_A_105.npy', '2_A_128.npy', '2_A_100.npy', '2_A_75.npy', '2_A_95.npy', '2_A_53.npy', '2_A_19.npy', '2_A_101.npy', '2_A_29.npy', '2_A_4.npy', '2_A_45.npy', '2_A_32.npy', '2_A_14.npy', '2_A_42.npy', '2_A_74.npy', '2_A_73.npy', '2_A_85.npy', '2_A_36.npy', '2_A_13.npy', '2_A_125.npy', '2_A_74.npy', '2_A_107.npy', '2_A_2.npy', '2_A_91.npy', '2_A_68.npy', '2_A_59.npy', '2_A_22.npy', '2_A_45.npy', '2_A_57.npy']


In [88]:
len(Z)

50